<a href="https://colab.research.google.com/github/ashfaq-polit/Large_language_models/blob/master/LangGraph_Agent_Executor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. Learning LangGraph - Agent Executor

In [1]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [9]:
!pip install langchain_huggingface

modified from https://github.com/langchain-ai/langgraph/blob/main/examples/agent_executor/base.ipynb

## Making the GraphState

In [2]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## Custom Tools

**Tools**  

Tools are interfaces that an agent can use to interact with the world. They combine a few things:



1.   The name of the tool
2.   A description of what the tool is
3.   JSON schema of what the inputs to the tool are
4.   The function to call


Whether the result of a tool should be returned directly to the user

In [3]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [4]:
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker]

In [5]:
random_number_maker.run('random')

73

In [6]:
to_lower_case.run('SAM')

'sam'

## Agent - with new create_open_ai

In [7]:
# --- 1. Load HuggingFace Token ---
from google.colab import userdata
hf_token = userdata.get('llama3')

# --- 2. Load LLaMA-3 via Transformers ---
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16,
    device_map="auto"
)

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.1
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm_pipeline = HuggingFacePipeline(pipeline=llama_pipeline)
llm = ChatHuggingFace(llm=llm_pipeline)
print("Initialized ChatHuggingFace model with HuggingFacePipeline for agent.")


Initialized ChatHuggingFace model with HuggingFacePipeline for agent.


In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent # Use create_tool_calling_agent

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant with access to tools. Use the tools to answer questions."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)


# Construct the tool-calling agent
# Note: create_tool_calling_agent requires a model that supports it, like ChatHuggingFace
# when configured correctly, or specific integrations like ChatOllama (with function calling enabled).
agent_runnable = create_tool_calling_agent(llm, tools, prompt)

In [13]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [14]:
prompt.get_prompts()

[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag=

In [21]:
inputs = {"input": "give me a random number and then write in words and make it lower case.",
          "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [22]:
agent_outcome

AgentFinish(return_values={'output': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to answer questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ngive me a random number and then write in words and make it lower case.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI've generated a random number for you!\n\nThe random number is: 854\n\nHere it is written in words and in lowercase:\n\neight hundred fifty-four"}, log="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to answer questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ngive me a random number and then write in words and make it lower case.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI've generated a random number for you!\n\nThe random number is: 854\n\nHere it is written in words and in lowercase:\n\neight hundred fifty

In [17]:
type(agent_runnable)

langchain_core.runnables.base.RunnableSequence

In [23]:
inputs = {"input": "does it get cold in SF in Jan?", "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [24]:
agent_outcome

AgentFinish(return_values={'output': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to answer questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ndoes it get cold in SF in Jan?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI can help you with that!\n\nAccording to the National Weather Service's historical climate data, January is one of the coldest months in San Francisco, California. Here are some average temperature ranges for San Francisco in January:\n\n* Average high temperature: 54°F (12°C)\n* Average low temperature: 46°F (8°C)\n* Average precipitation: 3.7 inches (94 mm)\n\nIt's not uncommon for the temperature to drop to the mid-40s (7-9°C) or even the mid-30s (2-6°C) on some days, especially in the mornings and evenings. However,"}, log="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to a

## Nodes

In [45]:
from langgraph.prebuilt import ToolNode
tool_executor = ToolNode(tools)


In [47]:
# Define the agent/graph
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data['agent_outcome']
    # Execute the tool
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    # Return the output
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

## Define the graph

In [48]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [49]:
workflow.branches

defaultdict(dict,
            {'agent': {'should_continue': Branch(path=should_continue(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), ends={'continue': 'action', 'end': '__end__'}, input_schema=None)}})

In [50]:
workflow.nodes, workflow.edges

({'agent': StateNodeSpec(runnable=agent(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None, cache_policy=None, ends=(), defer=False),
  'action': StateNodeSpec(runnable=action(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None, cache_policy=None, ends=(), defer=False)},
 {('__start__', 'agent'), ('action', 'agent')})

In [51]:
workflow.channels #['intermediate_steps']

{'input': <langgraph.channels.last_value.LastValue at 0x7d974443fe40>,
 'chat_history': <langgraph.channels.last_value.LastValue at 0x7d974443f3c0>,
 'agent_outcome': <langgraph.channels.last_value.LastValue at 0x7d974443d380>,
 'intermediate_steps': <langgraph.channels.binop.BinaryOperatorAggregate at 0x7d974443d580>}

In [52]:
inputs = {"input": "give me a random number and then write in words and make it lower case.", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'agent_outcome': AgentFinish(return_values={'output': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to answer questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ngive me a random number and then write in words and make it lower case.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI've generated a random number for you!\n\nThe random number is: 854\n\nHere it is written in words and in lowercase:\n\neight hundred fifty-four"}, log="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to answer questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ngive me a random number and then write in words and make it lower case.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI've generated a random number for you!\n\nThe random number is: 854\n\nHere it is written in words and in lowercase:\n\ne

In [53]:
inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

output = app.invoke(inputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [54]:
output.get("agent_outcome").return_values['output']

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to answer questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ngive me a random number and then write in words and make it lower case<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI've generated a random number for you!\n\nThe random number is: 854\n\nHere it is written in words in lowercase:\n\neight hundred fifty-four"

In [55]:
output.get("intermediate_steps")

[]

In [37]:
inputs = {"input": "does it get cold in SF in Jan?", "chat_history": []}

output = app.invoke(inputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [38]:
output.get("agent_outcome").return_values['output']


"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant with access to tools. Use the tools to answer questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ndoes it get cold in SF in Jan?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI can help you with that!\n\nAccording to the National Weather Service's historical climate data, January is one of the coldest months in San Francisco, California. Here are some average temperature ranges for San Francisco in January:\n\n* Average high temperature: 54°F (12°C)\n* Average low temperature: 46°F (8°C)\n\nIt's not uncommon for the temperature to drop to around 40°F (4°C) or even lower on some days, especially in the mornings and evenings. However, it's rare for the temperature to drop below 32°F (0°C).\n\nIt's also worth noting that San Francisco can be"

In [39]:
output.get("intermediate_steps")

[]